### 🧠 Building & Evaluating Complex Agents with `strands` and `flotorch-eval`

In this notebook, we'll walk through a complete example of evaluating agents using the **`flotorch-eval`** package across key metrics. These metrics help assess both agent quality and system performance.

---

#### 🔍 Evaluation Metrics

- **`AgentGoalAccuracyMetric`**  
  Evaluates whether the agent successfully understood and achieved the user's goal.  
  - **Binary** (1 = goal achieved, 0 = not achieved)

- **`ToolCallAccuracyMetric`**  
  Measures the correctness of tool usage by the agent—i.e., whether the agent called the right tools to complete a task.  
  - **Binary** (1 = relevant tools invoked, 0 = relevant tools not invoked)

- **`TrajectoryEvalWithLLM`**  
  Evaluates whether the trajectory (based on OpenTelemetry spans) is meaningful, either with or without a reference trajectory.  
  - **Binary** (1 = meaningful, 0 = invalid)

- **`LatencyMetric`**  
  Measures agent latency—how fast the agent responds or completes tasks.  
  

- **`UsageMetric`**  
  Evaluates the cost-efficiency of the agent in terms of compute, tokens, or other usage dimensions.  

---


#### Setup and dependencies

Installs `flotorch-eval`, `strands-agents`, and dependencies for developing Python agents and tools that execute and assess Python code.

Note: You might see some pip's dependency resolver issues - please ignore them.

In [1]:
# Install dependencies
!pip install numpy pandas -q
!pip install flotorch-eval uv -q

In [2]:
# Optionally install additional libraries that are needed for the use cases. 
# If you do not have them, LLM will identify missing libraries and will try to install in the Python REPL environment
!pip install yfinance matplotlib -q
!pip install beautifulsoup4 pandas requests -q
!pip install pandas numpy scikit-learn matplotlib seaborn joblib -q

In [ ]:
# Import libraries

import asyncio
import time
import pandas as pd
from typing import List

from strands import Agent, tool

from flotorch.strands.agent import FlotorchStrandsAgent

# Evaluation
from flotorch_eval.agent_eval.metrics.llm_evaluators import (
    TrajectoryEvalWithLLM, 
    TrajectoryEvalWithLLMWithReference, 
    ToolCallAccuracy, 
    AgentGoalAccuracy
    )
from flotorch_eval.agent_eval.metrics.usage_metrics import UsageMetric
from flotorch_eval.agent_eval.metrics.latency_metrics import LatencyMetric
from flotorch_eval.agent_eval.core.client import AgentEvaluator
from evaluation_utils import display_evaluation_results

#### Setting Up Tracing and Evaluation
To evaluate an agent, we first need to understand what it did. We'll use OpenTelemetry to "trace" the agent's execution. The code below sets up a custom QueueSpanExporter, which captures every step (span) of the agent's process and stores it in a queue for later analysis.

In [4]:
FLOTORCH_GATEWAY_BASE_URL = "https://dev-gateway.flotorch.cloud"
# FLOTORCH_API_KEY = "sk_XKtu651TxvB8C9/asbeXoi9DbuMKWlnSooh1Yt5sjr0=_YTBhZDdkNzYtNGZiZi00MzM4LThiZmQtZDFhNzE5NDZjNDNk_ZTkwN2RkYjEtNWYxYy00Y2ZiLTg3ZjktMWRlMTYyNGYyMmIw"
FLOTORCH_API_KEY="sk_OcQKFlEpA6UXX6fvhAOpR8chrHRQJdoiTk2kR/9b/D0=_YmFmNTNkZWYtM2ZjYi00M2RhLWE2MTYtODA4NGVkYTcwOWE1_OTVhMTk5YjktODdmYy00ZjYzLTg1ODktYmU4YjIxMzZhN2I0"

Finally, this asynchronous function ties everything together. It takes an agent, a prompt, and a list of metrics, then performs the entire run-trace-evaluate-display cycle

In [ ]:
async def evaluate_trajectory(trace_id, reference=None, reference_id=None):
    start_time = time.time()
    
    if reference and reference_id:
        raise ValueError("Provide either 'reference' or 'reference_trace_id', not both.")

    client = AgentEvaluator(
        api_key=FLOTORCH_API_KEY,
        base_url=FLOTORCH_GATEWAY_BASE_URL,
        default_evaluator="flotorch/haiku-long") # Setting a default evaluator for all metrics that require an LLM.

    traces = client.fetch_traces(trace_id)
    print(f"Traces: {traces}")
    results = await client.evaluate( # Metrics can be optionally provided as a list or a single metric
        trace=traces,
        reference=reference,
        reference_trace_id=reference_id
    )
    
    display_evaluation_results(results)
    print(results.model_dump_json(indent=4))
    end_time = time.time()
    time_taken = round(end_time - start_time, 2)
    print(f"Time taken for evaluation: {time_taken} seconds")

#### Use Case 1. Data frame manipulation agent

In [ ]:
dataframe_agent_factory = FlotorchStrandsAgent(
    agent_name="dataframe-agent",
    custom_tools=[]
)

dataframe_agent = dataframe_agent_factory.get_agent()

print("\n--- Agent Final Response ---")
response = dataframe_agent("Write a python script")


Evaluate the agent where no ground-truth answers or trajectories are provided. Instead, rely solely on an LLM-based evaluator to assess the agent's performance.

In [ ]:
trace_id = ""

async def main():
    await evaluate_trajectory(
        trace_id=trace_id
    )

await main()  

Evaluate the agent using reference data, including the expected agent response and trajectory. Use an LLM to compare the agent's output against this reference to assess performance.

In [ ]:
REFERENCE_TRAJECTORY = [
    {
        "input": "Write a Python script using the pandas library that performs the following tasks:\n\n- Create a sample DataFrame with the columns: 'Name', 'Age', and 'Salary'.\n- Add a new column named 'Bonus' that is 10% of the corresponding 'Salary' value.\n- Filter the DataFrame to include only rows where the 'Age' is greater than 30.\n- Group the data by age brackets (e.g., 20s, 30s, 40s) and calculate the average 'Salary' and 'Bonus' for each group.\n\nExecute the python script and show the output.\n\nRequirements:\n- Include clear inline comments to explain the logic.\n- Add a docstring for the function, describing its purpose, parameters, and return value.\n- Provide an example of how to use the function.\n- List any external libraries that need to be installed with pip (if any).\n- Include brief documentation describing how the code works and how to run it.",
        "expected_steps": [
            {
                "thought": "The user wants a Python script to perform several data manipulation tasks on tabular data. The pandas library is the ideal tool for this. I will structure the solution within a function to meet the reusability and documentation requirements.",
                "tool_call": "null"
            },
            {
                "thought": "I need to break down the user's request into four main pandas operations: 1) DataFrame creation from a dictionary. 2) Adding a new column via a simple vectorized calculation. 3) Filtering rows using boolean indexing. 4) Grouping by age brackets, which is best handled with `pd.cut` for binning, followed by `groupby().mean()` for the final aggregation. I will also plan out the documentation, comments, and example usage as requested.",
                "tool_call": "null"
            },
            {
                "thought": "I have planned all the components. I will now assemble the complete response, including the fully commented Python script, the documentation on how it works and how to run it, the installation requirements, the script's execution output, and the final trajectory detailing this thought process.",
                "final_response": "..."
            }
        ]
    }
]


In [ ]:
trace_id = ""

async def main():
    await evaluate_trajectory(
        trace_id=trace_id,
        reference=REFERENCE_TRAJECTORY
    )

await main()  

### Use Case 2: Financial advisor orchestrator

In [6]:
# Define the tools necessary for the agents

# Investment Research Assistant and it's tool to get the stock prices
@tool
def investment_research_assistant(query: str) -> str:
    investment_researcher_factory = FlotorchStrandsAgent(
        agent_name="loanparser",
    )
    return investment_researcher_factory.get_agent()(query)

# Budget Optimizer Assistant

@tool
def budget_optimizer_assistant(query: str) -> str:
    budget_coach_factory = FlotorchStrandsAgent(
        agent_name="creditanalyzer",
    )
    return budget_coach_factory.get_agent()(query)

# Financial Planner Assistant
@tool
def financial_planner_assistant(query: str) -> str:
    financial_planner_factory = FlotorchStrandsAgent(
        agent_name="riskassessor",
    )
    return financial_planner_factory.get_agent()(query)

In [7]:
# Define orchestrator system prompt with clear tool selection guidance


financial_orchestrator_factory = FlotorchStrandsAgent(
    agent_name="test",
    custom_tools=[investment_research_assistant, budget_optimizer_assistant, financial_planner_assistant]
)

# Define the complex user query
user_query_financial = """
    I'm 30 years old, earning around $6,000 per month. 
    I have some student loans and moderate savings.
    I want to understand how I can better manage my monthly budget, 
    explore investment options, and build a solid long-term financial plan 
    for buying a house and retiring early. Can you help?
    """

financial_orchestrator = financial_orchestrator_factory.get_agent()
financial_orchestrator(user_query_financial)

2025-10-08 10:35:18 - flotorch.sdk.llm - INFO - FlotorchLLM initialized (model_id=flotorch/gpt-4o-mini:latest, base_url=https://dev-gateway.flotorch.cloud)
2025-10-08 10:35:18 - flotorch.strands.llm - INFO - FlotorchStrandsModel initialized (model_id=flotorch/gpt-4o-mini:latest, base_url=https://dev-gateway.flotorch.cloud)
2025-10-08 10:35:18 - strands.telemetry.metrics - INFO - Creating Strands MetricsClient
2025-10-08 10:35:18 - flotorch.strands.utils.strands_utils - INFO - [Strands Integration] Starting stream with 1 messages
2025-10-08 10:35:18 - flotorch.strands.utils.strands_utils - INFO - [Strands Integration] Converted 1 messages to Flotorch format
2025-10-08 10:35:18 - flotorch.strands.utils.strands_utils - INFO - [Strands Integration] Converted tool: investment_research_assistant
2025-10-08 10:35:18 - flotorch.strands.utils.strands_utils - INFO - [Strands Integration] Converted tool: budget_optimizer_assistant
2025-10-08 10:35:18 - flotorch.strands.utils.strands_utils - INFO 

AgentResult(stop_reason='error', message={'role': 'assistant', 'content': []}, metrics=EventLoopMetrics(cycle_count=1, tool_metrics={}, cycle_durations=[5.3909077644348145], traces=[<strands.telemetry.metrics.Trace object at 0x72817b0903a0>], accumulated_usage={'inputTokens': 0, 'outputTokens': 0, 'totalTokens': 0}, accumulated_metrics={'latencyMs': 0}), state={})

Evaluate the agent where no ground-truth answers or trajectories are provided. Instead, rely solely on an LLM-based evaluator to assess the agent's performance.

In [ ]:
trace_id = ""

async def main():
    await evaluate_trajectory(
        trace_id=trace_id
    )

await main()  

In [ ]:
REFERENCE_TRAJECTORY = [
    {
        "input": "I'm 30 years old, earning around $6,000 per month. I have some student loans and moderate savings. I want to understand how I can better manage my monthly budget, explore investment options, and build a solid long-term financial plan for buying a house and retiring early. Can you help?",
        "expected_steps": [
            {
                "thought": "The user has three distinct financial goals: budgeting, investing, and long-term planning. I need to address each one methodically. The best starting point is to establish a solid budget, as this will determine the capital available for other goals. I will use the `budget_optimizer_assistant` for this first.",
                "tool_call": {
                    "name": "budget_optimizer_assistant",
                    "arguments": {
                        "query": "Create a personalized budget plan for a 30-year-old earning $6,000 per month with existing student loan debt and moderate savings."
                    }
                }
            },
            {
                "thought": "With the budgeting foundation laid, the next logical step is to address the user's request to explore investment options. Given their age and long-term goals (housing, early retirement), they can likely tolerate moderate risk for growth. I will use the `investment_research_assistant` to provide suitable options.",
                "tool_call": {
                    "name": "investment_research_assistant",
                    "arguments": {
                        "query": "Based on a profile of a 30-year-old with a long-term horizon, provide insights into appropriate investment opportunities including stocks, ETFs, and mutual funds."
                    }
                }
            },
            {
                "thought": "Now that I have handled the immediate budgeting and investment research, I need to synthesize this information into a cohesive, long-term strategy. This involves integrating debt management, savings, and investment timelines to create a clear roadmap for buying a house and retiring early. The `financial_planner_assistant` is designed for this comprehensive task.",
                "tool_call": {
                    "name": "financial_planner_assistant",
                    "arguments": {
                        "query": "Synthesize the budget and investment insights into a comprehensive long-term financial plan for a 30-year-old aiming to buy a house and retire early."
                    }
                }
            },
            {
                "thought": "I have gathered specialized advice on budgeting, investing, and long-term planning from all the relevant tools. My final step is to consolidate these outputs into a single, actionable, and encouraging summary for the user, outlining the key steps they can start taking today.",
                "final_response": "Great! Based on a full analysis, here is a step-by-step financial plan to help you manage your budget, invest wisely, and work towards buying a house and retiring early. First, let's look at your budget..."
            }
        ]
    }
]


Evaluate the agent using reference data, including the expected agent response and trajectory. Use an LLM to compare the agent's output against this reference to assess performance.

In [ ]:
trace_id = ""

async def main():
    await evaluate_trajectory(
        trace_id=trace_id,
        reference=REFERENCE_TRAJECTORY
    )

await main()  